In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [2]:
# from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *

In [3]:
# Source csv obtained from https://stockmarketmba.com/listofadrs.php
df = pd.read_csv("List_of_ADRs.csv")
# Filter for ADRs which have non-overlapping market hours, and those that we have IB data for
df = df.dropna(subset = ["Underlying Exchange"])
df = df[df["Country"].isin(["Australia", "China", "Japan"])]
df = df[~df["Underlying Exchange"].isin(["London Stock Exchange"])]
# Further filter for those that we can find on IB manually
df = df[~df["Symbol"].isin(["BHP", "BNTC", "SNE"])]
df.sort_values(["Country"])

,Symbol,Name,Country,Underlying Symbol,Underlying Exchange,GICS Sector,Market cap,P/E,P/S,P/B,Current yield,Avg Volume,Action
107,JHX,James Hardie Industries ADRs,Australia,AT:JHX,Asx - All Markets,Materials,"$14,421,616,850",84.29,5.29,11.29,0.00%,"28,094",Analyze
403,GENE,Genetic Technologies ADRs,Australia,AT:GTG,Asx - All Markets,Health Care,"$64,920,433",0.00,"6,623.62",4.99,0.00%,"552,907",Analyze
362,KZIA,Kazia Therapeutics ADRs,Australia,AT:KZA,Asx - All Markets,Health Care,"$158,070,642",0.00,"4,883.81",4.99,0.00%,"131,442",Analyze
350,IMMP,Immutep Limited ADRs,Australia,AT:IMM,Asx - All Markets,Health Care,"$210,448,464",0.00,21.19,3.47,0.00%,"6,207,627",Analyze
252,PLL,Piedmont Lithium Limited ADRs,Australia,AT:PLL,Asx - All Markets,Materials,"$1,047,422,124",-602.67,0.00,13.11,0.00%,"1,476,404",Analyze
251,MESO,Mesoblast ADRs,Australia,AT:MSB,Asx - All Markets,Health Care,"$1,071,645,908",-10.92,65.15,2.07,0.00%,"189,714",Analyze
409,ATHE,Alterity Therapeutics Limited ADRs,Australia,AT:PBT,Asx - All Markets,Health Care,"$50,363,736",0.00,"1,427.09",1.47,0.00%,"294,308",Analyze
424,IMRN,Immuron Limited ADRs,Australia,AT:IMC,Asx - All Markets,Health Care,"$34,086,990",0.00,13.94,1.19,0.00%,"22,810",Analyze
32,WBK,Westpac Banking Corp ADRs,Australia,AT:WBC,Asx - All Markets,Financials,"$69,776,616,188",39.04,20.85,1.03,1.24%,"107,977",Analyze
205,CEA,China Eastern Airlines ADRs,China,HK:0670,Hong Kong Exchange,Industrials,"$2,452,757,322",14.99,0.21,0.04,1.50%,"15,004",Analyze


Here are the list of ADR-stock pairs that are candidates for our trading strategy

In [4]:
#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)

history = '6 Y'
freq = '1 day'
side ='Trades'

In [5]:
# Pull data for candidate adr-stock pairs
# Change this cell to define different adr-stock pairs

adr_symbol = "CAJ"
adr_exchange = 'SMART'
adr_currency = "USD"

underlying_symbol = '7751'
underlying_exchange = 'TSEJ'
underlying_currency = 'JPY'
underlying_country = "Japan"

folder_name = underlying_country + "/" + adr_symbol + "_" + underlying_symbol + "/"

In [6]:
adr_contract = Contract(symbol = adr_symbol, secType = 'STK', exchange = adr_exchange, currency = adr_currency)
ib.qualifyContracts(adr_contract)
adr_df = get_data(adr_contract, history, freq, side)
print(adr_df.head(3))
# Uncomment this to overwrite existing data
# adr_df.to_csv(folder_name + "adr.csv")

         date   open   high    low  close  volume  average  barCount
0  2015-04-27  37.46  37.48  36.84  36.95    2443  37.0820      1696
1  2015-04-28  36.57  36.65  36.40  36.53    1694  36.5265      1275
2  2015-04-29  36.39  36.43  36.11  36.30    1567  36.2640      1227


In [7]:
underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
ib.qualifyContracts(underlying_contract)
underlying_df = get_data(underlying_contract, history, freq, side)
print(underlying_df.head(3))
# Uncomment this to overwrite existing data
# underlying_df.to_csv(folder_name + "underlying.csv")

         date    open    high     low   close   volume   average  barCount
0  2015-04-27  4510.0  4522.0  4469.0  4505.5  3728100  4501.045     10154
1  2015-04-28  4435.0  4440.0  4350.0  4358.5  7323100  4393.995     14425
2  2015-04-30  4300.0  4341.5  4254.5  4283.0  5390400  4303.245     12103


For pulling minute resolution data for stocks close

In [4]:
history = '40 D'
freq = '1 min'
side ='Trades'

aus_adr_underlying_pairs = [
    ["ATHE", "ATH"],["GENE", "GTG"],["IMMP", "IMM"],["IMRN","IMC"],["JHX", "JHX"],["KZIA", "KZA"],
    ["MESO","MSB"],["PLL", "PLL"],["WBK","WBC"]
]

hk_adr_underlying_pairs = [
    ["ACH", "2600"],["BGNE", "6160"],["CEA","670"],["HNP", "902"],["LFC", "2628"],["PTR", "857"],
    ["SHI", "338"],["SNP","386"],["ZNH", "1055"]
]

jpn_adr_underlying_pairs = [
    ["CAJ", "7751"],["HMC","7267"],["IX","8591"],["MFG","8411"],["MUFG","8306"],["NMR","8604"],
    ["SMFG","8316"],["SONY","6758"],["TAK", "4502"],["TM", "7203"]
]

country_list = [
    (aus_adr_underlying_pairs, "AUD", "SMART", "Australia" ),
    (hk_adr_underlying_pairs, "HKD", "SEHK", "China"),
    (jpn_adr_underlying_pairs, "JPY", "TSEJ", "Japan")
]



In [ ]:
date_pointer = datetime(2015,10,18)
end_date = datetime.now()
while date_pointer < end_date:
    
    print(date_pointer.date())
    debug_start_time = datetime.now()
    
    pull_until = date_pointer + timedelta(days = 40)
    
    for pairs, currency, exchange, country in country_list:
        # Pull data from ADRs in "Asian_ADRs.csv"
        print(country)
        for pair in pairs:
#             print(pair)
            adr_symbol = pair[0]
            adr_exchange = 'SMART'
            adr_currency = "USD"

            underlying_symbol = pair[1]
            underlying_exchange = exchange
            underlying_currency = currency
            underlying_country = country

            folder_name = underlying_country + "/" + adr_symbol + "_" + underlying_symbol + "/minute_data/"
            
            adr_contract = Contract(symbol = adr_symbol, secType = 'STK', exchange = adr_exchange, currency = adr_currency)
            ib.qualifyContracts(adr_contract)
            try:
                adr_df = get_data(adr_contract, history, freq, side, date_pointer)
    #             print(adr_df.head(1))
                adr_df.to_csv(folder_name + "adr_"+date_pointer.strftime("%Y%m%d")+"-"
                            +pull_until.strftime("%Y%m%d")+".csv")
            except:
                print("No Data for "+pair[0])
                
            underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
            ib.qualifyContracts(underlying_contract)
            try:
                underlying_df = get_data(underlying_contract, history, freq, side, date_pointer)
    #             print(underlying_df.head(1))
                underlying_df.to_csv(folder_name + "underlying_"+date_pointer.strftime("%Y%m%d")+"-"
                            +pull_until.strftime("%Y%m%d")+".csv")
            except:
                print("No Data for "+pair[1])
        
    date_pointer += timedelta(days = 40)
    debug_end_time = datetime.now()
    time_taken = debug_end_time - debug_start_time
    print(time_taken)
    

2015-10-18
Australia


Error 162, reqId 4: Historical Market Data Service error message:HMDS query returned no data: ATHE@SMART Trades, contract: Contract(secType='STK', conId=360308242, symbol='ATHE', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='ATHE', tradingClass='SCM')


No Data for ATHE


Error 162, reqId 12: Historical Market Data Service error message:HMDS query returned no data: IMMP@SMART Trades, contract: Contract(secType='STK', conId=298284081, symbol='IMMP', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='IMMP', tradingClass='NMS')


No Data for IMMP


...

No Data for IMRN


Error 162, reqId 368: Historical Market Data Service error message:HMDS query returned no data: PLL@SMART Trades, contract: Contract(secType='STK', conId=295822042, symbol='PLL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='PLL', tradingClass='SCM')


No Data for PLL
China


Error 162, reqId 382: Historical Market Data Service error message:HMDS query returned no data: 6160@SEHK Trades, contract: Contract(secType='STK', conId=328844678, symbol='6160', exchange='SEHK', primaryExchange='SEHK', currency='HKD', localSymbol='6160', tradingClass='6160')


No Data for 6160


In [ ]:
### Date controls for forex data pulling ###
def AUS_opening(date_now):
#     aus_dst = {
#         2021: (date(2021, 4, 4), date(2021, 10, 3)),
#         2020: (date(2020, 4, 5), date(2020, 10, 4)),
#         2019: (date(2019, 4, 7), date(2019, 10, 6)),
#         2018: (date(2018, 4, 1), date(2018, 10, 7)),
#         2017: (date(2017, 4, 2), date(2017, 10, 1)),
#         2016: (date(2016, 4, 3), date(2016, 10, 2)),
#         2015: (date(2015, 4, 5), date(2015, 10, 4)),
#     }
    
    us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    }
    
    us_start, us_end = us_dst[date_now.year]
    if us_start < date_now and date_now < us_end:
        return datetime.combine(date_now,time(10,0)) - timedelta(hours = 14)
    else:
        return datetime.combine(date_now, time(10,0)) - timedelta(hours = 15)

def HK_opening(date_now):
    us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    }
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,30)) - timedelta(hours = 12)
    else:
        return datetime.combine(date_now, time(9,30)) - timedelta(hours = 13)
    
def JPN_opening(date_now):
    us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    }
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 13)
    else:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 14)

For pulling forex data at minute resolution

In [6]:
forex_pairs = [["AUD",  "USD", "USD"],["USD", "HKD", "JPY",]]
opening_hours = [AUS_opening, HK_opening, JPN_opening]
df_dict = {}
forex_contracts = []
day_block = 5
history = str(day_block) + ' D'
freq = '1 min'
side ='BID_ASK'

In [1]:
date_pointer = datetime(2020,5,21)
end_date = datetime.now()

while date_pointer < end_date:    
    debug_start_time = datetime.now()
    
    start_date = date_pointer
    close_date = min(date_pointer + timedelta(days = 28),end_date)

    # Initialize Dictionary to store dataframes and initialize all the contracts
    df_dict = {}
    forex_contracts = []
    
    for i in range(3):
        currency1 = forex_pairs[0][i]
        secType = "CASH"
        currency2 = forex_pairs[1][i]
        exchange = "IDEALPRO"

        df_dict[currency1+"."+currency2] = []

        forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
        ib.qualifyContracts(forex_contract)
        forex_contracts.append(forex_contract)

    # Iterate over the datesace
    while date_pointer < close_date:
        print(date_pointer.date())

#         if date_pointer.weekday() >5: 
#             date_pointer += timedelta(days = 1)
#             continue

        for i in range(3):
            currency1 = forex_pairs[0][i]
            currency2 = forex_pairs[1][i]
            contract = forex_contracts[i]

            # NYSE Opening 
            enddate = date_pointer
            forex_df_1 = get_data(contract, history, freq, side, enddate)
            df_dict[currency1+"."+currency2].append(forex_df_1)
            ib.sleep(1)

#             # Other Exchange Opening
#             enddate = opening_hours[i](date_pointer) + timedelta(minutes = 2)
#             forex_df_2 = get_data(contract, history, freq, side, enddate)
#             df_dict[currency1+"."+currency2].append(forex_df_2)

        date_pointer += timedelta(days = day_block)
        

    folder_name = "Forex/"
    for i in range(3):
        currency1 = forex_pairs[0][i]
        currency2 = forex_pairs[1][i]
        df_forex = pd.concat(df_dict[currency1+"."+currency2], ignore_index = True,verify_integrity = True).sort_values(by = "date")
        df_forex.to_csv(folder_name + currency1 + "_" + currency2 + "_minute_"+start_date.strftime("%Y%m%d")+"-"
                        +close_date.strftime("%Y%m%d")+".csv")

    debug_end_time = datetime.now()
    time_taken = debug_end_time - debug_start_time
    print(time_taken)

Pull forex data but only the five minutes around each market open

In [7]:
forex_pairs = [["AUD",  "USD", "USD"],["USD", "HKD", "JPY",]]
opening_hours = [AUS_opening, HK_opening, JPN_opening]
df_dict = {}
forex_contracts = []
history = '300 S'
freq = '1 min'
side ='BID_ASK'

In [10]:
date_pointer = datetime(2020,5,21,9,30)
end_date = datetime.now()

while date_pointer < end_date:    
    debug_start_time = datetime.now()
    
    start_date = date_pointer
    close_date = min(date_pointer + timedelta(days = 30),end_date)

    # Initialize Dictionary to store dataframes and initialize all the contracts
    df_dict = {}
    forex_contracts = []
    
    for i in range(3):
        currency1 = forex_pairs[0][i]
        secType = "CASH"
        currency2 = forex_pairs[1][i]
        exchange = "IDEALPRO"

        df_dict[currency1+"."+currency2] = []

        forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
        ib.qualifyContracts(forex_contract)
        forex_contracts.append(forex_contract)

    # Iterate over the datesace
    while date_pointer < close_date:
        print(date_pointer.date())

        if date_pointer.weekday() >5: 
            date_pointer += timedelta(days = 1)
            continue

        for i in range(3):
            currency1 = forex_pairs[0][i]
            currency2 = forex_pairs[1][i]
            contract = forex_contracts[i]

            # NYSE Opening 
            enddate = date_pointer + timedelta(minutes = 2)
            forex_df_1 = get_data(contract, history, freq, side, enddate)
            df_dict[currency1+"."+currency2].append(forex_df_1)
            ib.sleep(1)

            # Other Exchange Opening
            enddate = opening_hours[i](date_pointer) + timedelta(minutes = 2)
            forex_df_2 = get_data(contract, history, freq, side, enddate)
            df_dict[currency1+"."+currency2].append(forex_df_2)
            ib.sleep(1)

        date_pointer += timedelta(days = 1)
        

    folder_name = "Forex/"
    for i in range(3):
        currency1 = forex_pairs[0][i]
        currency2 = forex_pairs[1][i]
        df_forex = pd.concat(df_dict[currency1+"."+currency2], ignore_index = True,verify_integrity = True).sort_values(by = "date")
        df_forex.to_csv(folder_name + currency1 + "_" + currency2 + "_minute_"+start_date.strftime("%Y%m%d")+"-"
                        +close_date.strftime("%Y%m%d")+".csv")

    debug_end_time = datetime.now()
    time_taken = debug_end_time - debug_start_time
    print(time_taken)

2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06
2020-06-07
2020-06-08
2020-06-09
2020-06-10
2020-06-11
2020-06-12
2020-06-13
2020-06-14
2020-06-15
2020-06-16
2020-06-17
2020-06-18
2020-06-19
0:11:55.043870
2020-06-20
2020-06-21
2020-06-22
2020-06-23
2020-06-24
2020-06-25
2020-06-26
2020-06-27
2020-06-28
2020-06-29
2020-06-30
2020-07-01
2020-07-02
2020-07-03
2020-07-04
2020-07-05
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-12
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-18
2020-07-19
0:11:59.979623
2020-07-20
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-25
2020-07-26
2020-07-27
2020-07-28
2020-07-29
2020-07-30
2020-07-31
2020-08-01
2020-08-02
2020-08-03
2020-08-04
2020-08-05
2020-08-06
2020-08-07
2020-08-08
2020-08-09
2020-08-10
2020-08-11
2020-08-12
2020-08-13
2020-08-14
2020-08-15
2020-08-16
20

Peer closed connection


In [21]:
df_forex

,date,open,high,low,close,volume,average,barCount
5,2021-03-14 19:57:00,109.119,109.122,109.117,109.121,-1,-1.0,-1
6,2021-03-14 19:58:00,109.117,109.125,109.110,109.119,-1,-1.0,-1
7,2021-03-14 19:59:00,109.113,109.120,109.110,109.115,-1,-1.0,-1
8,2021-03-14 20:00:00,109.119,109.127,109.114,109.120,-1,-1.0,-1
9,2021-03-14 20:01:00,109.088,109.117,109.074,109.090,-1,-1.0,-1
0,2021-03-14 23:57:00,109.130,109.137,109.124,109.131,-1,-1.0,-1
1,2021-03-14 23:58:00,109.128,109.133,109.125,109.129,-1,-1.0,-1
2,2021-03-14 23:59:00,109.122,109.127,109.119,109.123,-1,-1.0,-1
3,2021-03-15 00:00:00,109.125,109.128,109.124,109.127,-1,-1.0,-1
4,2021-03-15 00:01:00,109.131,109.134,109.125,109.132,-1,-1.0,-1
